# RAVE
**RAVE** is a variational autoencoder for fast and high-quality neural audio synthesis by Antoine Caillon and Philippe Esling.
\
\
[Article on arxiv](https://arxiv.org/abs/2111.05011) & [Source code on Github](https://github.com/acids-ircam/RAVE)

\
---

<small>*This notebook has been written to set up a training with RAVE on kaggle.com using GPUs. Persistence should be set to "Variables and Files"*
\
*Last updated: 10.03.2023*

# Install Conda, RAVE, dependencies

In [ ]:
#Install Conda
!curl -L https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh -o miniconda.sh
!chmod +x miniconda.sh
!sh miniconda.sh -b -p /kaggle/working/miniconda

#Install RAVE
#Pin to specific release tag if necessary using "acids-rave==tag.version"
!/kaggle/working/miniconda/bin/pip install acids-rave==2.1.12

#Upgrade ipython ipykernel, install ffmpeg
!/kaggle/working/miniconda/bin/pip install --upgrade pytorch_lightning ipython ipykernel
!/kaggle/working/miniconda/bin/conda install ffmpeg --yes

#Uninstall Cublas library. For some reason this creates conflicts with the environment on Kaggle. 
!/kaggle/working/miniconda/bin/pip uninstall nvidia_cublas_cu11 --yes

# Preprocess the dataset and start initial training

Preprocessing is necessary once per training set. In below section, the metadata from preprocessing is stored in '/kaggle/working/meta'. Use the filepath to your dataset on Kaggle as input path, this is usually '/kaggle/input/<yourdataset>'. 

While training the output is stored in a folder named <yourtrainingname> followed by an underscore and a 10-character-string inside '/kaggle/working/training/runs'. '/kaggle/working/training' also contains a 'status' folder with 'data.mdb' and 'lock.mdb' files.

In [ ]:
#Preprocess dataset
#Use flags "--lazy" when training on compressed dataset in various formats.
%cd /kaggle/working
!mkdir meta

#Optional: you can resample your material to a fixed sample rate. Files will be store in a out_<selectedsamplerate> subfolder. Make sure to change your dataset folder in the section above and rerun it. 
#samplerate = "44100" #@param ["22050", "44100", "48000"]
#resample --sr $samplerate --augment

!/kaggle/working/miniconda/bin/rave preprocess --input_path /kaggle/input/<yourdataset>/ --output_path /kaggle/working/meta/

#Initiate training
%cd /kaggle/working
!mkdir training
%cd /kaggle/working/training

#Use config parameters as preferred for your training. The setup below is exemplary.
!/kaggle/working/miniconda/bin/rave train --config v2.gin --config wasserstein --db_path /kaggle/working/meta/ --name <yourtrainingname> --override PHASE_1_DURATION=100000

# Resume training
In order to resume training RAVE on Kaggle, there are a few steps to take: 
1. Download **all relevant data** from your earlier training run you want to resume on (you can find this on the **output** tab)
2. Create a **new dataset** and upload all data while **maintaining the folder structure** ('meta', 'runs' and 'status' folders incl. subfolders)
3. Add dataset to the **notebook**. 
4. **Disable** the above initial **training section** of the notebook.

***For initial training, the below section has been disabled, enable when you want to resume your training and disable the initial training section before running the notebook.***

In [ ]:
##Copy contents of earlier training to /kaggle/working folder.
#!cp -r /kaggle/input/<rootfolderofyourearliertraining>/* /kaggle/working

#%cd /kaggle/working/training

##Resume training
##Use config parameters as used in your earlier training.
#!/kaggle/working/miniconda/bin/rave train --config v2.gin --config wasserstein --db_path /kaggle/working/meta/ --name <yourtrainingname> --override PHASE_1_DURATION=100000 --ckpt /kaggle/working/training/runs/<yourtrainingnamewitharandomstringintheend>/version_<XX>/checkpoints/

# Create model
After your training is finished, you can create a model (.ts Torchscript file). 

In [ ]:
#Model export. Use '--streaming' flag to export a model capable of real time processing.

#!/kaggle/working/miniconda/bin/rave export --run /kaggle/working/training/runs/<yourtrainingnamewitharandomstringintheend> --streaming